In [1]:
import tensorflow as tf
import os

# Define constants
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
DATA_DIR = 'chest_xray'

# Create training and validation datasets from the directory structure
train_dataset = tf.keras.utils.image_dataset_from_directory(
    os.path.join(DATA_DIR, 'train'),
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='binary',
    shuffle=True
)

validation_dataset = tf.keras.utils.image_dataset_from_directory(
    os.path.join(DATA_DIR, 'val'),
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='binary',
    shuffle=False
)

# Normalize the image data to be between 0 and 1
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))
validation_dataset = validation_dataset.map(lambda x, y: (normalization_layer(x), y))

print("Datasets created and preprocessed successfully.")

Found 0 files belonging to 0 classes.


ValueError: When passing `label_mode="binary"`, there must be exactly 2 class_names. Received: class_names=[]

In [2]:
# Load the pre-trained MobileNetV2 model without its top classification layer
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)

# Freeze the layers of the base model so they don't get retrained
base_model.trainable = False

# Create the final model by adding our custom classification layers on top
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid') # Sigmoid for binary (Normal/Pneumonia) classification
])

# Compile the model with an optimizer, loss function, and metrics
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Print a summary of the model architecture
model.summary()

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 20s 2us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,259,265 (8.62 MB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [3]:
# Train the model for a few epochs
# Note: This step will take some time to run
history = model.fit(
    train_dataset,
    epochs=5, # 5 epochs is a good balance of speed and performance for a hackathon
    validation_data=validation_dataset
)

# Save the final trained model to the backend folder
model.save('../backend/pneumonia_model.h5')

print("\n--- Training complete! ---")
print("Model saved as 'pneumonia_model.h5' in the 'backend' folder.")

NameError: name 'train_dataset' is not defined